In [3]:
import pandas as pd
from pandasql import sqldf
import pyparsing as pyp
from os import path
import string

# TODO Solve this problem with the relative paths.
DATA_IN = '../../data/clean/Jefferson_County_KY_Street_Intersections.csv'
assert path.exists(DATA_IN)

df = pd.read_csv(DATA_IN, index_col=0)
df.head(5)

,intid,main_street,cross_street,main_suffix,cross_suffix,cross_prefix,main_prefix,longitude,latitude
OBJECTID,,,,,,,,,
1,154647662,REHL,REHL,RD,CT,W,,-85.510444,38.205887
2,254646551,REHL,TUCKER STATION,RD,RD,,,-85.528169,38.200376
3,354646551,REHL,TUCKER STATION,RD,RD,,,-85.528419,38.200360
4,431949996,I 64 EAST,I 265 RAMP,,,,,-85.504954,38.222603
5,593499996,I 265 NORTH,I 265 RAMP,,,,,-85.505546,38.222127


In [33]:
MST = df.main_street

# Interstates
Iroads = MST[MST.str.startswith('I ')].unique()

number = pyp.Word(pyp.nums)
I_expr = pyp.Literal("I") + pyp.Word(pyp.nums) + pyp.Word(pyp.alphas.upper())

test = Iroads[0]

I, number, suffix  = I_expr.parse_string(test)
I, number, suffix

('I', '64', 'EAST')

In [36]:
XST = df.cross_street

XST[XST.str.startswith('I')].unique()

array(['I 265 RAMP', 'I 64 RAMP', 'ISLEWORTH', 'IAN', 'IVY OAKS',
       'I 71 RAMP', 'I 71 SOUTH', 'I 264 RAMP', 'INDIAN HILLS',
       'I 264 WEST', 'INNISBROOK', 'INNES TRACE', 'INDIAN RIDGE',
       'INDIAN WOODS', 'IOLA', 'IRIS', 'INDOCIN', 'INDIAN CREST',
       'ISABEL', 'INGLE', 'IRIS BED', 'INVERARY', 'INDIAN LEGENDS',
       'INDIAN LAKE', 'IVYBRIDGE', 'IRON GATE', 'ITWORTH', 'IVYWOOD',
       'INDIAN', 'ILEX', 'IDLEWOOD', 'IRVIN PINES', 'INDEPENDENCE SCHOOL',
       'IONA', 'IRON MOUNTAIN', 'INDIAN FALLS', 'IRONWOOD', 'IONIC',
       'I 65 RAMP', 'IDLEWYLDE', 'INDUSTRY', 'ILLINOIS', 'IOWA', 'IVY',
       'INDIAN CREEK', 'INDIANA', 'ISAAC', 'INNIS', 'INGLESIDE',
       'ISAAC CALDWELL', 'IVANHOE', 'INDIAN OAKS', 'IROQUOIS',
       'INVERNESS', 'IROQUOIS GARDENS', 'IDA', 'IVAN', 'INDEPENDENCE',
       'IVY RIDGE', 'INTERCHANGE', 'INDUSTRIAL', 'IDLE HOUR', 'IMPERIAL',
       'INVICTA', 'INDIGO', 'INTERNATIONAL', 'INTERMODAL', 'IRON HORSE',
       'INSPIRATION', 'ICHABOD', 'INTE

In [133]:
import re
re.search('[0-9]', '2nd')

<re.Match object; span=(0, 1), match='2'>

In [168]:

pattern = '([^0-9]*)([0-9]+)([^0-9]*)'
digit = '[0-9]'

def hasdigit(value):
    match = re.search(digit, value)
    if bool(match):
        return True
    else:
        return False
    
def startswith(value):
    match = re.search('^[0-9]', value)
    if not match:
        return False
    else:
        return match.string
    
startswith('3hello')


'3hello'

In [195]:
    
XS = df[df.cross_street.apply(hasdigit) == True].cross_street
hasdigit_names = pd.DataFrame(XS.unique(), columns=['raw'])
NST = hasdigit_names.raw.unique()

array(['I 265 RAMP', 'I 64 RAMP', 'HIGHWAY 22', 'I 71 RAMP', 'I 71 SOUTH',
       'I 264 RAMP', 'U S HIGHWAY 42', 'I 264 WEST', 'KY-841 RAMP',
       'HIGHWAY 42', 'HIGHWAY 329', 'HIGHWAY 1694', 'HIGHWAY 148', '38TH',
       'I 65 RAMP', '27TH', '22ND', '39TH', '35TH', '34TH', '29TH',
       '28TH', '45TH', '30TH', '44TH', '41ST', '40TH', '43RD', '37TH',
       '36TH', '42ND', '19TH', '26TH', '33RD', '32ND', '25TH', '20TH',
       '21ST', '23RD', '24TH', '17TH', '18TH', '16TH', '12TH', '13TH',
       '15TH', '31ST', '12TH STREET CONNECTOR', '11TH', '9TH', '3RD',
       '7TH', '2ND', '8TH', '4TH', '1ST', '5TH', '10TH', '6TH',
       '9TH STREET', '7TH STREET', 'KFEC GATE4', 'KFEC GATE3',
       'KFEC GATE2', 'KFEC GATE6', '22ND ST CONNECTOR', '46TH', '47TH',
       '3RD STREET', 'SYLVANIA NO 6', 'SYLVANIA NO 4', 'KY 841',
       'I 71 NORTH', '9TH STREET RAMP', 'KY-841', 'I 64 EAST',
       'I 264 EAST', 'I 265 SOUTH', 'I 64 WEST', 'I 265 NORTH',
       'I 65 SOUTH', 'I 65 NORTH', 'IN-2

In [213]:

prefixes = ("I", "K", "U", "HIGHWAY", "SYLVANIA")
filtered = [name for name in NST if not name.startswith(prefixes)]
filtered.sort()
filtered

name_expr  = pyp.Word(pyp.nums) + pyp.ZeroOrMore(pyp.Word(pyp.printables))
name_expr.parse_string('9TH And Fk').as_list()


['9', 'TH', 'And', 'Fk']

In [221]:
x, *y = [name_expr.parse_string(name) for name in filtered][14]
x,y

('22', ['ND', 'ST', 'CONNECTOR'])

In [240]:

def do(name):
    number, *coda = name_expr.parse_string(name)
    C1, *C2 = coda
    return number, C1, C2

out = pd.DataFrame.from_records([do(name) for name in filtered],
                          columns = 'number C1 coda'.split())
out.number = out.number.astype(int)
out.sort_values('number')

,number,C1,coda
10,1,ST,[]
22,2,ND,[]
34,3,RD,[STREET]
33,3,RD,[]
43,4,TH,[]
44,5,TH,[]
45,6,TH,[]
47,7,TH,[STREET]
46,7,TH,[]
48,8,TH,[]
